In [2]:
# Import dependencies
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.llms import Ollama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore # Import PineconeVectorStore
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

/Users/mohammedkhan/miniconda3/envs/rag-workshop/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [3]:
# Split file into chunks
with open("reviews.txt") as f:
    cardata = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
docs = text_splitter.create_documents([cardata])
print("Number of documents:", len(docs))

Number of documents: 465


In [4]:
# Generate embeddings & store on Pinecone
embeddings = OpenAIEmbeddings()
index_name = "cargpt"
vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)
retriever = vectorstore.as_retriever()

In [5]:
# Use an existing index on Pinecone
index_name = "cargpt"
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [6]:
# Prompt Template 

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
{question}
{context}
  """

prompt = PromptTemplate(
    input_variables=["context", "question"], 
    template=prompt_template,
  )

In [7]:
# Build the RAG Chain & Invoke

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#llm = Ollama(model="mistral")
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Tell me about Creta")

"The Creta is an SUV that is easy to drive and user-friendly. It has good visibility, comfortable ride, and handles well at higher speeds. It has a competitive boot volume of 433 litres and offers features like a powered driver's seat, ventilated front seats, dual-zone climate control, panoramic sunroof, and a 10.25-inch digital display. In terms of safety, it comes with 6 airbags, ABS with EBD, electronic stability control, and advanced driver assistance systems like lane keep assist and adaptive cruise control. The Creta also has drive modes for different driving preferences."